In [1]:
from sqlalchemy import ForeignKey, select, delete, String, and_
from sqlalchemy.orm import Session
from databaseold import MarkedSentence, LessonSession, engine

2024-09-12 20:19:08,480 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-12 20:19:08,480 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("word_dict")
2024-09-12 20:19:08,481 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-09-12 20:19:08,481 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("word_index")
2024-09-12 20:19:08,481 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-09-12 20:19:08,482 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("sentences")
2024-09-12 20:19:08,482 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-09-12 20:19:08,482 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("marked_sentences")
2024-09-12 20:19:08,483 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-09-12 20:19:08,483 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("lesson_sessions")
2024-09-12 20:19:08,483 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-09-12 20:19:08,483 INFO sqlalchemy.engine.Engine COMMIT


In [2]:
stmt = select(LessonSession)
with Session(engine) as session:
    for entry in session.scalars(stmt):
        print(entry)

2024-09-12 20:19:11,557 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-12 20:19:11,577 INFO sqlalchemy.engine.Engine SELECT lesson_sessions.id, lesson_sessions.lesson, lesson_sessions.date_time, lesson_sessions.errors_number 
FROM lesson_sessions
2024-09-12 20:19:11,578 INFO sqlalchemy.engine.Engine [generated in 0.00096s] ()
date time : datetime.datetime(2024, 9, 4, 22, 42, 30, 759328), lesson 50 errors number : 6
date time : datetime.datetime(2024, 9, 4, 23, 2, 0, 780033), lesson 1 errors number : 0
date time : datetime.datetime(2024, 9, 5, 9, 19, 11, 922138), lesson 51 errors number : 3
date time : datetime.datetime(2024, 9, 6, 23, 54, 35, 589923), lesson 3 errors number : 3
date time : datetime.datetime(2024, 9, 6, 23, 30, 29, 589456), lesson 52 errors number : 5
date time : datetime.datetime(2024, 9, 7, 15, 38, 11, 304630), lesson 53 errors number : 4
date time : datetime.datetime(2024, 9, 7, 15, 47, 54, 852500), lesson 4 errors number : 0
date time : datetime.datetime(202

In [3]:
stmt = select(LessonSession)
with Session(engine) as session:
    for entry in session.scalars(stmt):
        for sentence in entry.errored_sentences:
            print(sentence.session)
            

2024-09-11 22:31:46,172 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-11 22:31:46,174 INFO sqlalchemy.engine.Engine SELECT lesson_sessions.id, lesson_sessions.lesson, lesson_sessions.date_time, lesson_sessions.errors_number 
FROM lesson_sessions
2024-09-11 22:31:46,174 INFO sqlalchemy.engine.Engine [cached since 1.826s ago] ()
2024-09-11 22:31:46,180 INFO sqlalchemy.engine.Engine SELECT marked_sentences.id AS marked_sentences_id, marked_sentences.sentence AS marked_sentences_sentence, marked_sentences.comment AS marked_sentences_comment, marked_sentences.line AS marked_sentences_line, marked_sentences.session_id AS marked_sentences_session_id 
FROM marked_sentences 
WHERE ? = marked_sentences.session_id
2024-09-11 22:31:46,181 INFO sqlalchemy.engine.Engine [generated in 0.00111s] (1,)
date time : datetime.datetime(2024, 9, 4, 22, 42, 30, 759328), lesson 50 errors number : 6
date time : datetime.datetime(2024, 9, 4, 22, 42, 30, 759328), lesson 50 errors number : 6
date time : d

In [10]:
from database import get_lesson_sessions_history

In [11]:
sessions = get_lesson_sessions_history()

----------------- begin_date : None
----------------- end_date : None
2024-09-12 18:05:41,428 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-12 18:05:41,432 INFO sqlalchemy.engine.Engine SELECT lesson_sessions.id, lesson_sessions.lesson, lesson_sessions.date_time, lesson_sessions.errors_number 
FROM lesson_sessions 
WHERE lesson_sessions.date_time >= ? AND lesson_sessions.date_time <= ? AND lesson_sessions.lesson <= ? AND lesson_sessions.lesson >= ?
2024-09-12 18:05:41,433 INFO sqlalchemy.engine.Engine [cached since 7.043e+04s ago] ('0001-01-01 00:00:00.000000', '9999-12-31 23:59:59.999999', 100, 1)
2024-09-12 18:05:41,446 INFO sqlalchemy.engine.Engine ROLLBACK


In [12]:
sessions

{50: {datetime.datetime(2024, 9, 4, 22, 42, 30, 759328): 6},
 1: {datetime.datetime(2024, 9, 4, 23, 2, 0, 780033): 0},
 51: {datetime.datetime(2024, 9, 5, 9, 19, 11, 922138): 3},
 3: {datetime.datetime(2024, 9, 6, 23, 54, 35, 589923): 3},
 52: {datetime.datetime(2024, 9, 6, 23, 30, 29, 589456): 5},
 53: {datetime.datetime(2024, 9, 7, 15, 38, 11, 304630): 4},
 4: {datetime.datetime(2024, 9, 7, 15, 47, 54, 852500): 0},
 54: {datetime.datetime(2024, 9, 9, 0, 33, 9, 558002): 4},
 5: {datetime.datetime(2024, 9, 9, 0, 37, 30, 688736): 0},
 55: {datetime.datetime(2024, 9, 9, 23, 49, 20, 733862): 8},
 56: {datetime.datetime(2024, 9, 10, 11, 28, 32, 681006): 10},
 7: {datetime.datetime(2024, 9, 10, 11, 34, 52, 105091): 2},
 57: {datetime.datetime(2024, 9, 11, 15, 24, 26, 952928): 7}}

In [7]:
import datetime

In [9]:
stmt = select(LessonSession)
with Session(engine) as session:
    session.execute(delete(LessonSession).where(LessonSession.date_time >= datetime.datetime(2024, 9, 11, 21, 0, 0, 0)))
    session.commit()

2024-09-12 18:02:55,234 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-12 18:02:55,247 INFO sqlalchemy.engine.Engine DELETE FROM lesson_sessions WHERE lesson_sessions.date_time >= ?
2024-09-12 18:02:55,248 INFO sqlalchemy.engine.Engine [cached since 7.014e+04s ago] ('2024-09-11 21:00:00.000000',)
2024-09-12 18:02:55,253 INFO sqlalchemy.engine.Engine COMMIT


In [9]:
html_with_selection = """<p> N52-Lec<b>ción</b> cin<b>cuen</b>ta y dos </p>                  <p> S00-TITLE-La co<b>ci</b>na mexi<b>ca</b>na </p>                  <p> S01-No me has <b>di</b>cho <b>na</b>da de la co<b>ci</b>na mexi<b>ca</b>na </p>                  <p> S02-¿O a lo me<b>jor</b> no te atre<b>vis</b>te a pro<b>bar</b>la?  </p>                  <p> S03-¡Por su<b>pues</b>to que la pro<b>bé</b>! Me gus<b>tó</b> <b>so</b>bre <b>to</b>do el <b>mo</b>le es <b>u</b>na <b>sal</b>sa de ca<b>ca</b>o y guin<b>di</b>llas.  </p>                  <p> S04-¿Choco<b>la</b>te pi<b>can</b>te? ¿No te <mark class='V' title='3ème personne du singulier du passé simple de dar : dio et non pas di.'>dio</mark> <b>as</b>co?  </p>                  <p> S05-¡Qué va! ¡Es<b>tá</b> muy <b>ri</b>co! Lo a<b>mar</b>go va muy bien con lo pi<b>can</b>te.  </p>                  <p> S06-Los <b>pue</b>blos precolom<b>bi</b>nos ha<b>cí</b>an <b>u</b>na be<b>bi</b>da <b>ro</b>ja con el ca<b>ca</b>o.  </p>                  <p> S07-¡Por <b>e</b>so los conquista<b>do</b>res cre<b>ye</b>ron que be<b>bí</b>an <b>san</b>gre!  </p>                  <p> S08-¿<b>Pe</b>ro <b>nun</b>ca lo <mark class='V' title='3ème personne du pluriel du présent du verbe tomar.'><b>to</b>man</mark> <b></b><b>dul</b>ce?  </p>                  <p> S09-Sí, <b>cla</b>ro, hoy en <b>dí</b>a tam<b>bién</b> <mark class='G' title='cacao est complément d&#39;objet indirect donc="le" et non pas "lo"'>le</mark> <b>e</b>chan a<b>zú</b>car,  </p>                  <p> S10-<b>pe</b>ro en la tradi<b>ción</b> in<b>dí</b>gena no se <mark class='G' title='cacao est complément d&#39;objet indirect donc="le" et non pas "lo"'>le</mark> a<b>ña</b>de <b>le</b>che, <b>so</b>lo <b>a</b>gua.  </p>                  <p> S11-<b></b><mark class='O'><b></b><mark class='O' title='Meriendas est féminin, donc nuestras et non pas nuestros.'><b>Nues</b>tras</mark></mark> me<b>rien</b>das sí que las e<b>cha</b>ba de <b>me</b>nos un buen choco<b>la</b>te con <b>chu</b>rros. . .  </p>                  <p> S12-<b>E</b>so, <b>pa</b>ra meren<b>dar</b>, ¡<b>na</b>da <b>co</b>mo Es<b>pa</b>ña!  </p>                  <p> T00-TRANSLATE-Ejer<b>ci</b>cio 1 – Tra<b>duz</b>ca </p>                  <p> T01-No me has <b>di</b>cho si te gus<b>tó</b> la <b>sal</b>sa de ca<b>ca</b>o y guin<b>di</b>llas.  </p>                  <p> T02-¿Te atre<b>vis</b>te a to<b>mar</b> choco<b>la</b>te a<b>mar</b>go, pi<b>can</b>te y sin a<b>zú</b>car?  </p>                  <p> T03-¡Qué va! Me dio <b>as</b>co y no lo pro<b>bé</b>.  </p>                  <p> T04-Pues a lo me<b>jor</b> es<b>tá</b> muy <b>ri</b>co, <b>pe</b>ro pre<b>fie</b>ro un choco<b>la</b>te con <b>chu</b>rros.  </p>                  <p> T05-<b>E</b>cho de <b>me</b>nos la co<b>ci</b>na mexi<b>ca</b>na.  </p>"""

In [11]:
print(html_with_selection)

<p> N52-Lec<b>ción</b> cin<b>cuen</b>ta y dos </p>                  <p> S00-TITLE-La co<b>ci</b>na mexi<b>ca</b>na </p>                  <p> S01-No me has <b>di</b>cho <b>na</b>da de la co<b>ci</b>na mexi<b>ca</b>na </p>                  <p> S02-¿O a lo me<b>jor</b> no te atre<b>vis</b>te a pro<b>bar</b>la?  </p>                  <p> S03-¡Por su<b>pues</b>to que la pro<b>bé</b>! Me gus<b>tó</b> <b>so</b>bre <b>to</b>do el <b>mo</b>le es <b>u</b>na <b>sal</b>sa de ca<b>ca</b>o y guin<b>di</b>llas.  </p>                  <p> S04-¿Choco<b>la</b>te pi<b>can</b>te? ¿No te <mark class='V' title='3ème personne du singulier du passé simple de dar : dio et non pas di.'>dio</mark> <b>as</b>co?  </p>                  <p> S05-¡Qué va! ¡Es<b>tá</b> muy <b>ri</b>co! Lo a<b>mar</b>go va muy bien con lo pi<b>can</b>te.  </p>                  <p> S06-Los <b>pue</b>blos precolom<b>bi</b>nos ha<b>cí</b>an <b>u</b>na be<b>bi</b>da <b>ro</b>ja con el ca<b>ca</b>o.  </p>                  <p> S07-¡Por <b>e</

In [47]:
html_with_selection

'<p> N52-Lec<b>ción</b> cin<b>cuen</b>ta y dos </p>                  <p> S00-TITLE-La co<b>ci</b>na mexi<b>ca</b>na </p>                  <p> S01-No me has <b>di</b>cho <b>na</b>da de la co<b>ci</b>na mexi<b>ca</b>na </p>                  <p> S02-¿O a lo me<b>jor</b> no te atre<b>vis</b>te a pro<b>bar</b>la?  </p>                  <p> S03-¡Por su<b>pues</b>to que la pro<b>bé</b>! Me gus<b>tó</b> <b>so</b>bre <b>to</b>do el <b>mo</b>le es <b>u</b>na <b>sal</b>sa de ca<b>ca</b>o y guin<b>di</b>llas.  </p>                  <p> S04-¿Choco<b>la</b>te pi<b>can</b>te? ¿No te <mark class=\'V\' title=\'3ème personne du singulier du passé simple de dar : dio et non pas di.\'>dio</mark> <b>as</b>co?  </p>                  <p> S05-¡Qué va! ¡Es<b>tá</b> muy <b>ri</b>co! Lo a<b>mar</b>go va muy bien con lo pi<b>can</b>te.  </p>                  <p> S06-Los <b>pue</b>blos precolom<b>bi</b>nos ha<b>cí</b>an <b>u</b>na be<b>bi</b>da <b>ro</b>ja con el ca<b>ca</b>o.  </p>                  <p> S07-¡Por <

In [48]:
from selection import extract_selection

In [49]:
marked_lines_numbers, sentences = extract_selection(html_with_selection)

attr_name : class, attr_value : V
---- in convert_to_html_escape, returning V
---- escaped_attr_value : V
attr_name = class, char : V, 86, 0
attr_name : title, attr_value : 3ème personne du singulier du passé simple de dar : dio et non pas di.
---- in convert_to_html_escape, returning 3ème personne du singulier du passé simple de dar : dio et non pas di.
---- escaped_attr_value : 3ème personne du singulier du passé simple de dar : dio et non pas di.
attr_name = title, char : 3, 51, 0
attr_name = title, char : è, 232, 1
attr_name = title, char : m, 109, 2
attr_name = title, char : e, 101, 3
attr_name = title, char :  , 32, 4
attr_name = title, char : p, 112, 5
attr_name = title, char : e, 101, 6
attr_name = title, char : r, 114, 7
attr_name = title, char : s, 115, 8
attr_name = title, char : o, 111, 9
attr_name = title, char : n, 110, 10
attr_name = title, char : n, 110, 11
attr_name = title, char : e, 101, 12
attr_name = title, char :  , 32, 13
attr_name = title, char : d, 100, 14
attr

In [50]:
marked_lines_numbers

{5: True, 9: True, 10: True, 11: True, 12: True}

In [51]:
sentences

['N52-Lec<b>ción</b> cin<b>cuen</b>ta y dos',
 'S00-TITLE-La co<b>ci</b>na mexi<b>ca</b>na',
 'S01-No me has <b>di</b>cho <b>na</b>da de la co<b>ci</b>na mexi<b>ca</b>na',
 'S02-¿O a lo me<b>jor</b> no te atre<b>vis</b>te a pro<b>bar</b>la?',
 'S03-¡Por su<b>pues</b>to que la pro<b>bé</b>! Me gus<b>tó</b> <b>so</b>bre <b>to</b>do el <b>mo</b>le es <b>u</b>na <b>sal</b>sa de ca<b>ca</b>o y guin<b>di</b>llas.',
 "S04-¿Choco<b>la</b>te pi<b>can</b>te? ¿No te <mark class='V' title='3ème personne du singulier du passé simple de dar : dio et non pas di.'>dio</mark> <b>as</b>co?",
 'S05-¡Qué va! ¡Es<b>tá</b> muy <b>ri</b>co! Lo a<b>mar</b>go va muy bien con lo pi<b>can</b>te.',
 'S06-Los <b>pue</b>blos precolom<b>bi</b>nos ha<b>cí</b>an <b>u</b>na be<b>bi</b>da <b>ro</b>ja con el ca<b>ca</b>o.',
 'S07-¡Por <b>e</b>so los conquista<b>do</b>res cre<b>ye</b>ron que be<b>bí</b>an <b>san</b>gre!',
 "S08-¿<b>Pe</b>ro <b>nun</b>ca lo <mark class='V' title='3ème personne du pluriel du présent du verb

In [52]:
for index, sentence in enumerate(sentences):
    print(index, sentence)

0 N52-Lec<b>ción</b> cin<b>cuen</b>ta y dos
1 S00-TITLE-La co<b>ci</b>na mexi<b>ca</b>na
2 S01-No me has <b>di</b>cho <b>na</b>da de la co<b>ci</b>na mexi<b>ca</b>na
3 S02-¿O a lo me<b>jor</b> no te atre<b>vis</b>te a pro<b>bar</b>la?
4 S03-¡Por su<b>pues</b>to que la pro<b>bé</b>! Me gus<b>tó</b> <b>so</b>bre <b>to</b>do el <b>mo</b>le es <b>u</b>na <b>sal</b>sa de ca<b>ca</b>o y guin<b>di</b>llas.
5 S04-¿Choco<b>la</b>te pi<b>can</b>te? ¿No te <mark class='V' title='3ème personne du singulier du passé simple de dar : dio et non pas di.'>dio</mark> <b>as</b>co?
6 S05-¡Qué va! ¡Es<b>tá</b> muy <b>ri</b>co! Lo a<b>mar</b>go va muy bien con lo pi<b>can</b>te.
7 S06-Los <b>pue</b>blos precolom<b>bi</b>nos ha<b>cí</b>an <b>u</b>na be<b>bi</b>da <b>ro</b>ja con el ca<b>ca</b>o.
8 S07-¡Por <b>e</b>so los conquista<b>do</b>res cre<b>ye</b>ron que be<b>bí</b>an <b>san</b>gre!
9 S08-¿<b>Pe</b>ro <b>nun</b>ca lo <mark class='V' title='3ème personne du pluriel du présent du verbe tomar.'><b>to</b

In [15]:
stmt = select(MarkedSentence)
with Session(engine) as session:
    for entry in session.scalars(stmt):
        print(entry)


2024-09-12 18:15:40,450 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-12 18:15:40,453 INFO sqlalchemy.engine.Engine SELECT marked_sentences.id, marked_sentences.sentence, marked_sentences.comment, marked_sentences.line, marked_sentences.session_id 
FROM marked_sentences
2024-09-12 18:15:40,454 INFO sqlalchemy.engine.Engine [cached since 538.1s ago] ()
Sentence(id=1, line=2, sentence ="S01-Fe<b>li</b>pe, <b>gua</b>po, <b>da</b>me diez <b>lon</b>chas de <mark class='A'>ja<b>món</b></mark><b></b> de york, por fa<b>vor</b>.", comment=None)
Sentence(id=2, line=5, sentence ="S04-Y <mark class='E'>yo</mark> se la di a <b>es</b>ta se<b>ño</b>ra.", comment=None)
Sentence(id=3, line=6, sentence ="S05-O <b>se</b>a, que a<b>ho</b>ra me <b>to</b>ca a mí, y us<b>ted</b> ¡a ha<b>cer</b> <b>co</b>la i<b>gual</b> que <mark class='O'>los</mark> de<b>más</b>!", comment=None)
Sentence(id=4, line=10, sentence ="S09-<b></b><mark class='E'><b>Ven</b>ga</mark>, <b>pa</b>se, <b>pe</b>ro que <b>se</b>a

In [16]:
stmt = select(MarkedSentence)
with Session(engine) as session:
    for entry in session.scalars(stmt):
        try:
            print(entry.session.lesson)
        except:
            print("----entry without lesson : ", entry.id)
            session.execute(delete(MarkedSentence).where(MarkedSentence.id == entry.id))
    session.commit()

            #session.execute(delete(entry))
            

2024-09-12 18:15:43,376 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-12 18:15:43,378 INFO sqlalchemy.engine.Engine SELECT marked_sentences.id, marked_sentences.sentence, marked_sentences.comment, marked_sentences.line, marked_sentences.session_id 
FROM marked_sentences
2024-09-12 18:15:43,378 INFO sqlalchemy.engine.Engine [cached since 541s ago] ()
2024-09-12 18:15:43,383 INFO sqlalchemy.engine.Engine SELECT lesson_sessions.id AS lesson_sessions_id, lesson_sessions.lesson AS lesson_sessions_lesson, lesson_sessions.date_time AS lesson_sessions_date_time, lesson_sessions.errors_number AS lesson_sessions_errors_number 
FROM lesson_sessions 
WHERE lesson_sessions.id = ?
2024-09-12 18:15:43,384 INFO sqlalchemy.engine.Engine [cached since 541s ago] (1,)
50
50
50
50
50
50
2024-09-12 18:15:43,386 INFO sqlalchemy.engine.Engine SELECT lesson_sessions.id AS lesson_sessions_id, lesson_sessions.lesson AS lesson_sessions_lesson, lesson_sessions.date_time AS lesson_sessions_date_time, lesso

In [54]:
from selection import store_lesson_errors

In [55]:
store_lesson_errors(52, sentences, marked_lines_numbers, datetime.datetime(2024, 9, 6, 23, 30, 29, 589456))

-------------- In store_lesson_errors
---- 0 ------ N52-Lec<b>ción</b> cin<b>cuen</b>ta y dos
---- 1 ------ S00-TITLE-La co<b>ci</b>na mexi<b>ca</b>na
---- 2 ------ S01-No me has <b>di</b>cho <b>na</b>da de la co<b>ci</b>na mexi<b>ca</b>na
---- 3 ------ S02-¿O a lo me<b>jor</b> no te atre<b>vis</b>te a pro<b>bar</b>la?
---- 4 ------ S03-¡Por su<b>pues</b>to que la pro<b>bé</b>! Me gus<b>tó</b> <b>so</b>bre <b>to</b>do el <b>mo</b>le es <b>u</b>na <b>sal</b>sa de ca<b>ca</b>o y guin<b>di</b>llas.
---- 5 ------ S04-¿Choco<b>la</b>te pi<b>can</b>te? ¿No te <mark class='V' title='3ème personne du singulier du passé simple de dar : dio et non pas di.'>dio</mark> <b>as</b>co?
---- 6 ------ S05-¡Qué va! ¡Es<b>tá</b> muy <b>ri</b>co! Lo a<b>mar</b>go va muy bien con lo pi<b>can</b>te.
---- 7 ------ S06-Los <b>pue</b>blos precolom<b>bi</b>nos ha<b>cí</b>an <b>u</b>na be<b>bi</b>da <b>ro</b>ja con el ca<b>ca</b>o.
---- 8 ------ S07-¡Por <b>e</b>so los conquista<b>do</b>res cre<b>ye</b>ron que be

In [56]:
select

<function sqlalchemy.sql._selectable_constructors.select(*entities: '_ColumnsClauseArgument[Any]', **__kw: 'Any') -> 'Select[Any]'>

In [57]:
from database import MarkedSentence

In [24]:
stmt = select(LessonSession).where(
                        LessonSession.lesson == 58)

with Session(engine) as session:
    for entry in session.scalars(stmt):
        for sentence in entry.errored_sentences:
            print(sentence.id, sentence.session)


2024-09-12 18:23:06,707 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-12 18:23:06,731 INFO sqlalchemy.engine.Engine SELECT lesson_sessions.id, lesson_sessions.lesson, lesson_sessions.date_time, lesson_sessions.errors_number 
FROM lesson_sessions 
WHERE lesson_sessions.lesson = ?
2024-09-12 18:23:06,733 INFO sqlalchemy.engine.Engine [cached since 341.9s ago] (58,)
2024-09-12 18:23:06,736 INFO sqlalchemy.engine.Engine SELECT marked_sentences.id AS marked_sentences_id, marked_sentences.sentence AS marked_sentences_sentence, marked_sentences.comment AS marked_sentences_comment, marked_sentences.line AS marked_sentences_line, marked_sentences.session_id AS marked_sentences_session_id 
FROM marked_sentences 
WHERE ? = marked_sentences.session_id
2024-09-12 18:23:06,737 INFO sqlalchemy.engine.Engine [cached since 7.148e+04s ago] (15,)
58 date time : datetime.datetime(2024, 9, 12, 18, 13, 29, 359135), lesson 58 errors number : 4
59 date time : datetime.datetime(2024, 9, 12, 18, 13, 29

In [18]:
sfdb

[Sentence(id=58, line=7, sentence ="S06-De<b>cí</b>an que <b>e</b>ra entrete<b>ni</b>do, y me ha pare<b>ci</b>do <mark class='B'>aburri<b>dí</b>simo</mark>. ", comment=None),
 Sentence(id=59, line=9, sentence ="S08-¿Me es<b>tá</b> <mark class='O' title='&#034;gastando&#034; et non pas &#034;haciendo&#034;'>gas<b>tan</b>do</mark> <b>u</b>na <b>bro</b>ma? ", comment=None),
 Sentence(id=60, line=10, sentence ="S09-En abso<b>lu</b>to. <b>Cuan</b>do un apa<b>ra</b>to no fun<b>cio</b>na, <b>pi</b>de que se lo a<b>rre</b>glen o que le <mark class='V' title='&#034;devuelvan&#034; est au subjonctif'>de<b>vuel</b>van</mark> el di<b>ne</b>ro, ¿no? ", comment=None),
 Sentence(id=61, line=13, sentence ="S12-Soy muy cuida<b>do</b>so con los <b>li</b>bros, <mark class='E' title='&#034;incluso&#034; au lieu de &#034;hasta&#034;'>in<b>clu</b>so</mark> con los que no me <b>gus</b>tan. ", comment=None)]

In [20]:
sfdb[0].session


DetachedInstanceError: Parent instance <MarkedSentence at 0x108acfb50> is not bound to a Session; lazy load operation of attribute 'session' cannot proceed (Background on this error at: https://sqlalche.me/e/20/bhk3)

In [61]:
print(sfdb[2].sentence)

S09-Sí, <b>cla</b>ro, hoy en <b>dí</b>a tam<b>bién</b> <mark class='G' title='cacao est complément d&#039;objet indirect donc=&#034;le&#034; et non pas &#034;lo&#034;'>le</mark> <b>e</b>chan a<b>zú</b>car,


In [35]:
len(sfdb)

5

In [3]:
import database
from sqlalchemy import ForeignKey, select, delete, String, and_
from sqlalchemy.orm import Session
from database import MarkedSentence, LessonSession, User, engine

ImportError: cannot import name 'User' from 'database' (/Users/pam/assimil/database.py)

In [2]:
stmt = select(LessonSession).where(
                        LessonSession.lesson == 58)

with Session(engine) as session:
    for entry in session.scalars(stmt):
        for sentence in entry.errored_sentences:
            print(sentence.id, sentence.session)


NameError: name 'select' is not defined

In [1]:
import database
import databaseold
from sqlalchemy import ForeignKey, select, delete, String, and_
from sqlalchemy.orm import Session

2024-09-12 21:26:25,934 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-12 21:26:25,935 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("word_dict")
2024-09-12 21:26:25,935 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-09-12 21:26:25,936 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("word_index")
2024-09-12 21:26:25,936 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-09-12 21:26:25,936 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("sentences")
2024-09-12 21:26:25,936 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-09-12 21:26:25,936 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("marked_sentences")
2024-09-12 21:26:25,937 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-09-12 21:26:25,937 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2024-09-12 21:26:25,937 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-09-12 21:26:25,937 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("lesson_sessions")
2024-09-12 21:26:25,937 INFO sqlalche

In [2]:
from database import engine as engine_new

In [3]:
from databaseold import engine as engine_old

In [4]:
from database import User

In [5]:
with Session(engine_new) as session:
    pam = User(
        username="pam",
        password="TOTO1234",
        fullname="Pierre-Alain Moret",
        email="moret.pierre.alain@gmail.com"
    )
    session.add(pam)
    session.commit()

2024-09-12 21:10:55,477 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-12 21:10:55,479 INFO sqlalchemy.engine.Engine INSERT INTO user_account (username, password, fullname, email) VALUES (?, ?, ?, ?)
2024-09-12 21:10:55,480 INFO sqlalchemy.engine.Engine [generated in 0.00043s] ('pam', 'TOTO1234', 'Pierre-Alain Moret', 'moret.pierre.alain@gmail.com')
2024-09-12 21:10:55,481 INFO sqlalchemy.engine.Engine COMMIT


In [20]:
stmt = select(User)
with Session(engine_new) as session:
    for user in session.scalars(stmt):
        #for sentence in entry.errored_sentences:
            print(user.id, user.username, user.email)


2024-09-12 20:39:43,052 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-12 20:39:43,053 INFO sqlalchemy.engine.Engine SELECT user.id, user.username, user.password, user.fullname, user.email 
FROM user
2024-09-12 20:39:43,054 INFO sqlalchemy.engine.Engine [cached since 211.6s ago] ()
1 pam moret.pierre.alain@gmail.com
2024-09-12 20:39:43,056 INFO sqlalchemy.engine.Engine ROLLBACK


In [10]:
stmt_old = select(databaseold.LessonSession)
session_new = Session(engine_new) 
stmt_new = select(User).where(User.username == "pam")
pam = session.scalars(stmt_new).one()
print(pam)
with Session(engine_old) as session_old:
    for user_session in session_old.scalars(stmt_old):
        print(user_session.id)
        pam.lesson_sessions.append(database.LessonSession(
            user=pam,
            lesson = user_session.lesson,
            date_time = user_session.date_time
        ))
    session_new.commit()
    
            


2024-09-12 21:21:48,726 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.username, user_account.password, user_account.fullname, user_account.email 
FROM user_account 
WHERE user_account.username = ?
2024-09-12 21:21:48,735 INFO sqlalchemy.engine.Engine [cached since 641.4s ago] ('pam',)
username : 'pam', password : 'TOTO1234', full name :Pierre-Alain Moret
2024-09-12 21:21:48,752 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-12 21:21:48,754 INFO sqlalchemy.engine.Engine SELECT lesson_sessions.id, lesson_sessions.lesson, lesson_sessions.date_time, lesson_sessions.errors_number 
FROM lesson_sessions
2024-09-12 21:21:48,757 INFO sqlalchemy.engine.Engine [cached since 411s ago] ()
2024-09-12 21:21:48,760 INFO sqlalchemy.engine.Engine ROLLBACK


In [12]:
stmt_new = select(User).where(User.username == "pam")
pam = session.scalars(stmt_new).one()

pam
    


2024-09-12 21:22:13,757 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.username, user_account.password, user_account.fullname, user_account.email 
FROM user_account 
WHERE user_account.username = ?
2024-09-12 21:22:13,759 INFO sqlalchemy.engine.Engine [cached since 666.4s ago] ('pam',)


username : 'pam', password : 'TOTO1234', full name :Pierre-Alain Moret

In [11]:
stmt_new = select(database.LessonSession)
with Session(engine_new) as session_new:
    for user_session in session_new.scalars(stmt_new):
        print(user_session)

2024-09-12 21:22:05,455 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-12 21:22:05,457 INFO sqlalchemy.engine.Engine SELECT lesson_sessions.id, lesson_sessions.lesson, lesson_sessions.date_time, lesson_sessions.errors_number, lesson_sessions.user_id 
FROM lesson_sessions
2024-09-12 21:22:05,458 INFO sqlalchemy.engine.Engine [cached since 244.8s ago] ()
2024-09-12 21:22:05,460 INFO sqlalchemy.engine.Engine ROLLBACK


In [5]:
stmt_old = select(databaseold.LessonSession)
with Session(engine_old) as session_old:
    for lesson_session in session_old.scalars(stmt_old):
        print(lesson_session)

2024-09-12 21:26:43,398 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-12 21:26:43,414 INFO sqlalchemy.engine.Engine SELECT lesson_sessions.id, lesson_sessions.lesson, lesson_sessions.date_time, lesson_sessions.errors_number 
FROM lesson_sessions
2024-09-12 21:26:43,414 INFO sqlalchemy.engine.Engine [generated in 0.00078s] ()
2024-09-12 21:26:43,416 INFO sqlalchemy.engine.Engine ROLLBACK
